In [1]:
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, make_scorer, log_loss, mean_squared_error
from IPython.display import display_html
from copy import deepcopy
import pickle
from sklearn import calibration
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV, cross_val_predict, GridSearchCV
# from utils.utils

In [4]:
pd.set_option("display.max_columns", None)
model_df = pd.read_parquet("model_outputs_cfb.parquet")
input_names_score_diff_pred = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    # 'home_vegas_score_pred_weighted',
    # 'away_vegas_score_pred_weighted',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
    'kick_off',
    'point_after_play',
    'search_rf_play_first_down_home',
    'search_rf_play_first_down_away',
    'search_rf_drive_field_goal_made_home',
    'search_rf_drive_field_goal_missed_home',
    'search_rf_drive_punt_home',
    'search_rf_drive_safety_home',
    'search_rf_drive_touch_down_home',
    'search_rf_drive_turnover_home',
    'search_rf_drive_turnover_on_downs_home',
    'search_rf_drive_field_goal_made_away',
    'search_rf_drive_field_goal_missed_away',
    'search_rf_drive_punt_away',
    'search_rf_drive_safety_away',
    'search_rf_drive_touch_down_away',
    'search_rf_drive_turnover_away',
    'search_rf_drive_turnover_on_downs_away'
]



KeyboardInterrupt: 

In [9]:
list(model_df.columns)

['game_code',
 'game_date',
 'season',
 'home_team_id',
 'home_team',
 'home_team_abbrev',
 'home_conf_name',
 'home_division_type',
 'away_team_id',
 'away_team',
 'away_team_abbrev',
 'away_conf_name',
 'away_division_type',
 'home_final_score',
 'away_final_score',
 'final_score_diff',
 'end_of_regulation_score_diff',
 'home_rest_of_game_score',
 'away_rest_of_game_score',
 'end_of_regulation_score_diff_change',
 'home_score_added',
 'away_score_added',
 'current_score_diff',
 'current_score_total',
 'home_start_score',
 'away_start_score',
 'home_team_outcome',
 'home_team_win',
 'draw',
 'away_team_win',
 'nevent',
 'quarter',
 'overtime',
 'home_team_has_ball',
 'off_team_id',
 'def_team_id',
 'kick_off',
 'punt',
 'point_after_kick',
 'two_point_attempt',
 'field_goal_attempt',
 'off_start_score',
 'off_end_score',
 'off_score_change',
 'def_start_score',
 'def_end_score',
 'def_score_change',
 'play_counts',
 'efficiency_counts',
 'from_scrimmage',
 'first_down',
 'scoring_play

In [11]:
model_df

,time_left_in_half,half,current_score_diff,current_score_total,cur_spread,cur_over_under,home_timeouts_remaining,away_timeouts_remaining,ytg,yd_from_goal,down,home_team_has_ball,kick_off,point_after_play,search_rf_play_first_down_home,search_rf_play_first_down_away,search_rf_drive_field_goal_made_home,search_rf_drive_field_goal_missed_home,search_rf_drive_punt_home,search_rf_drive_safety_home,search_rf_drive_touch_down_home,search_rf_drive_turnover_home,search_rf_drive_turnover_on_downs_home,search_rf_drive_field_goal_made_away,search_rf_drive_field_goal_missed_away,search_rf_drive_punt_away,search_rf_drive_safety_away,search_rf_drive_touch_down_away,search_rf_drive_turnover_away,search_rf_drive_turnover_on_downs_away,event_name,xhome_win_mlp_search_clipped_rf_drive_preds
35,1203.0,1.0,-6,6,6.5,44.0,2,3,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
69,652.0,1.0,-4,16,6.5,44.0,2,1,-1,-1,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
167,691.0,2.0,-3,29,6.5,44.0,3,3,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
234,1421.0,1.0,-18,18,16.5,60.0,3,3,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
245,1272.0,1.0,-13,25,16.5,60.0,3,3,-1,-1,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699768,622.0,2.0,34,48,-9.5,47.0,3,2,-1,-1,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
1699784,221.0,2.0,29,55,-9.5,47.0,3,2,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
1699955,983.0,2.0,-3,21,3.0,53.0,3,3,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
1700012,NaN,2.0,-7,43,3.0,53.0,3,2,-1,-1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Point after Touchdown,NaN
